## Import Packages
We need to install transformers and datasets. soundfile is used to load audio files and jiwer is used to evaluate the finetuned model using word errot rate

In [ ]:
!pip install datasets>=1.18.3
!pip install transformers==4.11.3
!pip install librosa
!pip install jiwer
# Restart the runtime for this change to take effect
!pip install accelerate -U

To upload our training checkpoints directly to huggingface, we have to store the huggingface authentication key.

In [1]:
from huggingface_hub import notebook_login

notebook_login()

Install the GIT LFS in order to upload the model checkpoints

In [2]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


# Prepare Data, Tokenizer, Feature Extractor

### Create Wav2Vec2CTCTokenizer

In [3]:
# Load the dataset
from datasets import load_dataset, load_metric, Audio

# You can pass the streaming option to load_dataset to stream the data from the source instead of downloading and caching it
luganda = load_dataset("mozilla-foundation/common_voice_7_0", "lg")

print(luganda)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1454: FutureWarning: The repository for mozilla-foundation/common_voice_7_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_7_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next

DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 6626
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 4276
    })
    validation: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 3549
    })
    other: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 29407
    })
    invalidated: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 2195
    })
})


In [4]:
# Remove the unnecessary columns from the dataset
luganda = luganda.remove_columns(["client_id", "up_votes", "down_votes", "age", "gender", "accent", "locale", "segment"])

### Display some of the rows in the dataset

In [5]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML
import re

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(luganda["train"].remove_columns(["path", "audio"]))

,sentence
0,Pulojekiti zirina ebirubirirwa ne ssalessale.
1,"""Emmumbwa emu okuba ng'ejjanjaba endwadde nga kkumi kye kyannemesa okukkiririza mu kinnansi."""
2,Abavuzi ba ttakisi abasinga tebalina mpisa.
3,Waliwo obutakwatagana mu kusalawo ku disitulikiti.
4,Nali saagala busomesa naye bakadde bange baabunkaka okubusoma.
5,Ssentebe yatukubirizza okusimba emiti emirala.
6,Amasomero ga buseere okugayimirizaawo.
7,"""Minisitule y'ebyobulamu egemye abantu okusobola okuziyiza endwadde omukaaga zi nnamutta."""
8,Lwaki olowooza nti twetaaga okumanya ebintu ebikunyumira?
9,"""Abalamuzi abaawummula baayitibwa olw'emisango emingi egyali teginnakolwako mu minisitule."""


In [6]:
# Let's normalize the dataset to only lower case letters and ignore any special tokens because without a language model it is difficult to classify such tokens as they do not correspond to a characteristic sound.
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower()
    return batch

luganda = luganda.map(remove_special_characters)

In [7]:
# Display samples from the normalized dataset
show_random_elements(luganda["train"].remove_columns(["path", "audio"]))

,sentence
0,kcca ekoze ekisobooka okukungaanya kasasiro mu kampala
1,ssemateeka ejja kutumbula obumu mu bantu
2,omukyala oyo olabika nga omwagala nnyo
3,amasomero g'ebyobulimi gagguddwawo okuyambako okutumbula ebyobulimi
4,ekirwadde kya ssenyiga kisaasaanye nnyo mu mawanga g'abazungu
5,disitulikiti ejja kweyongera okuyamba mirimu gy'abantu okulongoosa embeera yaabwe
6,nnyumirwa nnyo okulya ettooke n'enkoko eya bboyiro
7,emiwendo gy'emisango egy'obuliisamaanyi gyeyongera nnyo
8,abavubuka basendebwasendebwa bannaabwe okwenyigira mu kukozesa ebiragalalagala
9,baliraanwa bange abawoggana bakosa obulamu bwange


In CTC chunks of speech are classified into letters. We need to extract all distinct letters in the dataset and builf a vocabulary.   
We need a mapping function that will concatenate all the transcriptions into a long transcription and transforms the strings into a set of characters.

In [8]:
# Let's use the batched = True so that the map function can access all the transcriptions at a go
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

vocabs = luganda.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=luganda.column_names["train"])

Map:   0%|          | 0/6626 [00:00<?, ? examples/s]

Map:   0%|          | 0/4276 [00:00<?, ? examples/s]

Map:   0%|          | 0/3549 [00:00<?, ? examples/s]

Map:   0%|          | 0/29407 [00:00<?, ? examples/s]

Map:   0%|          | 0/2195 [00:00<?, ? examples/s]

In [9]:
# Create a vocabulary of all letters in the train dataset
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'r': 0,
 'k': 1,
 'l': 2,
 'u': 3,
 'b': 4,
 'g': 5,
 'a': 6,
 't': 7,
 '(': 8,
 'm': 9,
 'z': 10,
 'i': 11,
 '’': 12,
 'w': 13,
 'o': 14,
 'x': 15,
 'y': 16,
 ' ': 17,
 'h': 18,
 ')': 19,
 'n': 20,
 'c': 21,
 'p': 22,
 "'": 23,
 's': 24,
 'f': 25,
 'e': 26,
 'd': 27,
 '‘': 28,
 'v': 29,
 'j': 30}

We need to replace the " " in the dataset with a more visible character. We also need to add the UNKNOWN token so that to deal with characters not encountered in the training dataset.

In [10]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

We need to add the pad token that corresponds to CTC's blank token. The blank token is a core component of the CTC algorithm.

In [11]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))

33


In [24]:
# Save the vocabulary to a json file
import json
with open('./vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [12]:
# Use the json file to instantiate an object of the Wav2Vec2CTCTokenizer class
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [35]:
# Push the tokenizer to the hub
repo_name = "luganda_wav2vec2_ctc_tokenizer"
tokenizer.push_to_hub(repo_name)

README.md:   0%|          | 0.00/2.44k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/dmusingu/luganda_wav2vec2_ctc_tokenizer/commit/b749503f876e21b02084b7968a94d20c6d82818d', commit_message='Upload tokenizer', commit_description='', oid='b749503f876e21b02084b7968a94d20c6d82818d', pr_url=None, pr_revision=None, pr_num=None)

# Create Wav2Vec Feature Extractor

In [14]:
# Create a feature extractor using Wav2Vec2FeatureExtractor. We shall pass feature size as 1 because we are dealing with raw audio files.
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [15]:
# Wrap the feature extractor and the tokenizer into a Wav2VecProcessor
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

# Preprocess the dataset

In [16]:
from datasets import Audio

In [17]:
luganda = luganda.cast_column("audio", Audio(sampling_rate=16000))

In [18]:
# Dislay an audio sample from the dataset
luganda['train'][10]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/f41fa3f1dcaf43ac5071b5022181b4a0d77f872115acbc3c8386d9568963751d/cv-corpus-7.0-2021-07-21/lg/clips/common_voice_lg_23722908.mp3',
 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         1.87113613e-09, -7.67451491e-10,  0.00000000e+00]),
 'sampling_rate': 16000}

In [19]:
# Listen to sample audio from the dataset
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(luganda["train"]))

print(luganda["train"][rand_int]["sentence"])
ipd.Audio(data=np.asarray(luganda["train"][rand_int]["audio"]["array"]), autoplay=True, rate=16000)

ekkanisa yayungudde abakuuma ddembe abalala


In [20]:
rand_int = random.randint(0, len(luganda["train"]))

print("Target text:", luganda["train"][rand_int]["sentence"])
print("Input array shape:", np.asarray(luganda["train"][rand_int]["audio"]["array"]).shape)
print("Sampling rate:", luganda["train"][rand_int]["audio"]["sampling_rate"])

Target text: omusomesa alina olusuku olunene
Input array shape: (79488,)
Sampling rate: 16000


In [21]:
# Convert the sampling frewquency to 16kHz since the model was pretrained on audio sampled at 16kHz
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=16000).input_values[0]

    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [22]:
# Apply the map function to the dataset
luganda = luganda.map(prepare_dataset, remove_columns=luganda.column_names["train"], num_proc=4)

### Training and Evaluation

In [23]:
# Set up the trainer
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [24]:
# Initialize the data_collator
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [25]:
# Load the metric
wer_metric = load_metric("wer")

<ipython-input-25-5217f441bba2>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:753: FutureWarning: The repository for wer contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.1/metrics/wer/wer.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [26]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [27]:
# Load the pretrained Wav2Vec2 checkpoint. We use the tokenizers pad token id to degine the model's pad token id
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base",
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
)

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:365: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.bias', 'lm_head.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weig

In [28]:
# Freeze the feature extractor
model.freeze_feature_extractor()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1919: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [ ]:
!pip install transformers[torch]

In [29]:
# Define the training arguments
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=32,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  gradient_checkpointing=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2
)

In [30]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=luganda["train"],
    eval_dataset=luganda["test"],
    tokenizer=processor.feature_extractor,
)

In [ ]:
# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
500,4.136500,1.959844,1.000000
1000,0.569500,0.585338,0.732862
1500,0.176000,0.538128,0.674720
2000,0.084500,0.512766,0.626989
2500,0.042400,0.465070,0.601355
3000,0.012700,0.539471,0.604891
3500,-0.006300,0.516856,0.584168
4000,-0.021200,0.499033,0.583284
4500,-0.033600,0.531802,0.568029
5000,-0.042400,0.546537,0.570156


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Tran

Step,Training Loss,Validation Loss,Wer
500,4.136500,1.959844,1.000000
1000,0.569500,0.585338,0.732862
1500,0.176000,0.538128,0.674720
2000,0.084500,0.512766,0.626989
2500,0.042400,0.465070,0.601355
3000,0.012700,0.539471,0.604891
3500,-0.006300,0.516856,0.584168
4000,-0.021200,0.499033,0.583284
4500,-0.033600,0.531802,0.568029
5000,-0.042400,0.546537,0.570156


In [32]:
# Push the model to hub
trainer.push_to_hub(repo_name)

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

events.out.tfevents.1708526749.30ee7847a043.7857.0:   0%|          | 0.00/12.2k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.73k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/dmusingu/luganda_wav2vec2_ctc_tokenizer/commit/841b4b6b64d379592f1cb8f9a059ab6139b30c28', commit_message='luganda_wav2vec2_ctc_tokenizer', commit_description='', oid='841b4b6b64d379592f1cb8f9a059ab6139b30c28', pr_url=None, pr_revision=None, pr_num=None)

### Evaluation

In [36]:
processor = Wav2Vec2Processor.from_pretrained(repo_name)
# model = Wav2Vec2ForCTC.from_pretrained(repo_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [39]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [40]:
# Evaluation is carried out with a batch size of 1
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)

  return batch

results = luganda["test"].map(map_to_result, remove_columns=luganda["test"].column_names)

Map:   0%|          | 0/4276 [00:00<?, ? examples/s]

In [41]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))

Test WER: 0.475


In [44]:
# Chech the errors made by the model
show_random_elements(results)

,pred_str,text
0,abazadde b'aba[UNK]izi be baagobaku ssomero bakkiriza okukozesa emeeza abaana ze baa[UNK]onoona,abazadde b'aba[UNK]izi be baagoba ku ssomero bakkirizza okukozesa emmeeza abaana ze ba[UNK]onoona
1,akaakanu kann[UNK]umidde nn[UNK]o bambi,ako akanu kakun[UNK]umidde nn[UNK]o bambi
2,abakulembeze ab'enjawulo beeab[UNK]e mu musomo,abakulembeze ab'enjawulo beeab[UNK]e mu musomo
3,im[UNK]obulwanga [UNK]amukui we esinga mu nabanda,iimu [UNK]a proline [UNK]'emu ku iimu ezisinga mu uganda
4,buzi bu ki obuba mu kufumbo obw'ekio,buzibu ki obuva mu bufumbo bw'ekio
5,omukulembeze omulungi [UNK]'o[UNK]o aegeera ebizibu b[UNK]abo baakulembera,omukulembeze omulungi [UNK]'o[UNK]o aegeera ebizibu b[UNK]'abo b'akulembera
6,maama wange ama[UNK]i okuboobeza emmere,maama wange aman[UNK]i okuboobeza emmere
7,ensi ezimu ze ugenda okukoleramu olwa oa [UNK]annungi mu ndabika na[UNK]e ebikole bwa[UNK]o b[UNK]a abpu,ensi ezimu ze ugenda okukoleramu obwa [UNK]aa[UNK]a nnungi mu ndabika na[UNK]e ebikolebwa[UNK]o b[UNK]a abu
8,eeampu zi realaba nu okuvukumi bifobira bwe erewagenda muolokippe ebi[UNK]emweddreeme,enalo zireeera abanu okuva mu bifo b[UNK]abwe ne bagenda mu bifo ebirimu eddembe
9,omuamiivu abeera nga e [UNK]eeba se na[UNK]e ngaai ngeera bulungi nn[UNK]a,omuamiivu abeera nga e[UNK]eebase na[UNK]e nga aegeera bulungi nn[UNK]o


From the output above we can make the following observations
1. The model is unable classifies the y's in the input as [UNK]. A similar observation can be made about the about the true values. This means that there is an error in the tokenization process.
2. The wav2vec model was pretrained on English which has a different morphology from Luganda and this could could be one of the possible causes of the high WER on the test set. More experiments need to be carried out to prove if this is the case.